# RLHF Trainining Pipeline for PPO Agent to Play Levels from the Doom Game

## Creating UI Elements

In [1]:
from ipywidgets import widgets
from typing import Callable
from datetime import datetime

# Function for rendering widgets
def create_html_heading(text, level=1, centered=True):
    html_tag = f'<h{level}>{text}</h{level}>'
    layout = widgets.Layout(display='flex', justify_content='center') if centered else None
    return widgets.HTML(html_tag, layout=layout)

def create_video_player(video_path: str):
    return widgets.Video.from_file(video_path)

def create_button(description, tooltip):
    return widgets.Button(description=description, disabled=False, button_style='',
                          tooltip=tooltip, layout=widgets.Layout(display='flex', justify_content='center', margin='4px'))

def create_loading_spinner():
    css = """
    .loader {
        border: 8px solid #f3f3f3;
        border-top: 8px solid #3498db;
        border-radius: 50%;
        width: 100px;
        height: 100px;
        animation: spin 2s linear infinite;
    }

    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    """
    # Create the custom CSS and apply it to the heading
    heading_style = widgets.HTML(f'<style>{css}</style>')
    # Create the loading spinner widget
    loading_spinner = widgets.HTML('<div class="loader"></div>', layout=widgets.Layout(display='flex', justify_content='center'))
    
    return heading_style, loading_spinner

def create_preference_selection_component(trajectory_1_video_path: str, trajectory_2_video_path: str, on_trajectory_1_chosen: Callable, on_trajectory_2_chosen: Callable, on_both_trajectories_chosen: Callable):
    # Creating GUI Components
    window_label = create_html_heading('Which trajectory do you prefer?', centered=True)
    trajectory_1_label = create_html_heading('Trajectory 1', level=2, centered=True)
    trajectory_2_label = create_html_heading('Trajectory 2', level=2, centered=True)
    trajectory_1_video_player = create_video_player(trajectory_1_video_path)
    trajectory_2_video_player = create_video_player(trajectory_2_video_path)
    prefer_trajectory_1_button = create_button('Select 1', 'You prefer Trajectory 1')
    prefer_trajectory_2_button = create_button('Select 2', 'You prefer Trajectory 2')
    prefer_both_trajectories_button = create_button('Both', 'You prefer both Trajectories')
    
    prefer_trajectory_1_button.on_click(on_trajectory_1_chosen)
    prefer_trajectory_2_button.on_click(on_trajectory_2_chosen)
    prefer_both_trajectories_button.on_click(on_both_trajectories_chosen)

    window_label.layout.flex = '1'

    # Rendering components
    preference_selection_layout = widgets.VBox([
        window_label,
        widgets.HBox([
            widgets.VBox([
                trajectory_1_label,
                trajectory_1_video_player,
                prefer_trajectory_1_button
            ], layout=widgets.Layout(align_items='center', justify_content='space-between', width='40%', height='100%')),
            widgets.VBox([
                trajectory_2_label,
                trajectory_2_video_player,
                prefer_trajectory_2_button
            ], layout=widgets.Layout(align_items='center', justify_content='space-between', width='40%', height='100%'))
        ], layout=widgets.Layout(justify_content='space-between', flex='4', width='100%')),
        widgets.HBox([
            prefer_both_trajectories_button
        ], layout=widgets.Layout(justify_content='space-between', flex='1'))
    ], layout=widgets.Layout(align_items='center', justify_content='space-between'))

    return preference_selection_layout

def create_loading_component(message):
    # Create the heading widget and apply the style
    heading = create_html_heading(message, centered=True)
    # Create the loading spinner widget
    heading_style, loading_spinner = create_loading_spinner()

    # Create the outer VBox layout with widgets
    outer_vbox = widgets.VBox([heading_style, heading, loading_spinner], layout=widgets.Layout(align_items='center', justify_content='center'))

    return outer_vbox

def create_logs_component():
    logs_label = widgets.HTML('<h1>Logs</h1>')
    logs_output = widgets.Output(description="Output")
    logs_layout = widgets.VBox([
            logs_label,
            widgets.Box([
                logs_output
            ], layout=widgets.Layout(flex='1', width='98%', overflow_y='scroll'))
        ]
    )

    return logs_layout, logs_output

logs_component, logs_output = create_logs_component()
layout = widgets.HBox([])

def log_info(message: str):
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    message = f"[{current_timestamp}][INFO] {message}"
    print(message)

def log_error(message: str):
    current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    message = f"[{current_timestamp}][ERROR] {message}\n"
    logs_output.append_stderr(message)

def hide_all_screens():
    layout.children = []

def show_preference_selection_screen(trajectory_1_video_path: str, trajectory_2_video_path: str, on_trajectory_1_chosen: Callable, on_trajectory_2_chosen: Callable, on_both_trajectories_chosen: Callable):
    preference_selection_component = create_preference_selection_component(trajectory_1_video_path, trajectory_2_video_path, on_trajectory_1_chosen, on_trajectory_2_chosen, on_both_trajectories_chosen)

    # Updating layout of components to fit the screen layout
    preference_selection_component.layout.padding = '15px' 
    preference_selection_component.layout.margin = '7.5px' 
    preference_selection_component.layout.border = '3px dashed cornflowerblue' 
    preference_selection_component.layout.flex = '1' 
    preference_selection_component.layout.height = '600px'
    
    logs_component.layout.padding = '15px'
    logs_component.layout.margin = '7.5px' 
    logs_component.layout.border = '3px dashed cornflowerblue' 
    logs_component.layout.flex = '1' 
    logs_component.layout.height = '600px'

    layout.children = [ preference_selection_component, logs_component ]

def show_loading_screen(message: str):
    loading_component = create_loading_component(message)

    # Updating layout of components to fit the screen layout
    loading_component.layout.padding = '15px'
    loading_component.layout.margin = '7.5px' 
    loading_component.layout.border = '3px dashed cornflowerblue' 
    loading_component.layout.flex = '1' 
    loading_component.layout.height = '600px'
    
    logs_component.layout.padding = '15px'
    logs_component.layout.margin = '7.5px' 
    logs_component.layout.border = '3px dashed cornflowerblue' 
    logs_component.layout.flex = '1' 
    logs_component.layout.height = '600px'

    layout.children = [ loading_component, logs_component ]

## Training Code

In [2]:
# from datetime import datetime

# start_datetime = datetime.now()
# start_datetime_timestamp_str = start_datetime.strftime("%Y_%m_%d_%H_%M_%S")

# # Agent training config
# start_time = start_datetime.timestamp()
# num_steps = 256
# num_mini_batches = 32
# num_training_epochs = 10
# num_envs = 8
# batch_size = int(num_envs * num_steps)
# mini_batch_size = batch_size // num_mini_batches

# # Reward predictor training config
# num_frames_in_single_trajectory_video = 64
# human_feedback_interval = 1 # Number of training iterations of PPO agent before asking for human feedback

In [3]:
import numpy as np
import time
from utils.video import generate_video_from_doom_play_segments
import random
import time
from IPython.display import display, clear_output
import asyncio
import traceback
from datetime import datetime
from utils.replay_buffer import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter
from agents.doom_ppo_agent import DoomPpoAgent
from reward_predictors.doom_human_preference_reward_predictor import DoomHumanPreferenceRewardPredictor
import gymnasium as gym

def ask_user_for_preference(trajectory_1_video_path: str, trajectory_2_video_path: str):
    future = asyncio.Future()

    def on_trajectory_1_chosen(_):
        future.set_result(0)
    
    def on_trajectory_2_chosen(_):
        future.set_result(1)
    
    def on_both_trajectories_chosen(_):
        future.set_result(0.5)

    show_preference_selection_screen(trajectory_1_video_path, trajectory_2_video_path, on_trajectory_1_chosen, on_trajectory_2_chosen, on_both_trajectories_chosen)

    return future

def create_random_pairs(elements):
    # Shuffle the elements array
    shuffled_elements = elements.copy()
    random.shuffle(shuffled_elements)
    
    # Create pairs of consecutive elements
    pairs = [(shuffled_elements[i], shuffled_elements[i+1]) for i in range(0, len(shuffled_elements), 2)]
    
    return pairs

async def pre_train(envs: gym.vector.SyncVectorEnv, 
                    agent: DoomPpoAgent, 
                    reward_predictor: DoomHumanPreferenceRewardPredictor, 
                    num_replay_buffer_steps_per_env: int, 
                    num_preference_requests: int,
                    num_steps_in_trajectory_video: int):
    # Creating replay buffer for storing transition for training
    replay_buffer = ReplayBuffer(num_replay_buffer_steps_per_env, 
                                 envs.num_envs, 
                                 envs.envs[0].raw_observation_space, 
                                 envs.single_observation_space, 
                                 envs.single_action_space)

    # Initializing pre-training variables 
    global_step = 0
    observations, infos = envs.reset()
    terminations = [0 for _ in range(envs.num_envs)]
    reward_sums = np.zeros(envs.num_envs, dtype=np.float32)
    current_num_preference_requests = 0

    # for i in range(num_preference_requests):
    while current_num_preference_requests < num_preference_requests:
        show_loading_screen(f"Exploring Enviroment to Pre-Train Reward Predictor...")
        
        for step in range(0, replay_buffer.num_steps):
            global_step += envs.num_envs

            # Getting next action and it's value
            actions, log_probs, probs, values = agent.forward(observations)
            values = values.flatten()
            
            # Performing actions in the environments
            observations_, _, terminations_, _, infos = envs.step(actions)

            # Predicting reward for the observations and the corresponding actions
            rewards = reward_predictor.forward(observations)
            reward_sums = reward_sums + rewards

            # Saving transitions in replay buffer
            replay_buffer[step] = (
                np.stack(infos["raw_observations"]),
                observations,
                actions,
                log_probs,
                rewards,
                values,
                terminations
            )

            # Saving new observation and termination status for next step
            observations = observations_
            terminations = terminations_

        # Preparing trajectories and videos for asking user for preference
        show_loading_screen("Preparing trajectories for asking user for preference...")
        segments = replay_buffer.get_segments(segment_length=num_steps_in_trajectory_video)
        trajectory_pairs = create_random_pairs(segments)

        for trajectory_1, trajectory_2 in trajectory_pairs:
            generate_video_from_doom_play_segments(trajectory_1, './temp/trajectory_1.mp4')
            generate_video_from_doom_play_segments(trajectory_2, './temp/trajectory_2.mp4')
            
            # Asking user for their preference
            user_preference = await ask_user_for_preference('./temp/trajectory_1.mp4', './temp/trajectory_2.mp4')
            log_info(f"user_preference = {user_preference}")

            # Training reward predictor based on the user preference
            show_loading_screen("Pre-training reward predictor...")
            try:
                reward_predictor_training_stats = reward_predictor.train(trajectory_1, trajectory_2, user_preference)
                log_info(f"reward_predictor_training_loss={reward_predictor_training_stats['loss']}")
                current_num_preference_requests += 1
            except Exception as error:
                # Catch the error and print the entire stack trace
                traceback.print_exc()

async def train(envs: gym.vector.SyncVectorEnv, 
                agent: DoomPpoAgent, 
                reward_predictor: DoomHumanPreferenceRewardPredictor,
                num_replay_buffer_steps_per_env: int,
                num_agent_training_mini_batches: int,
                num_agent_training_epochs: int,
                num_steps_in_trajectory_video: int,
                save_training_stats: bool = False):
    # Setting up other training config 
    start_time = start_datetime.timestamp()
    batch_size = int(envs.num_envs * num_replay_buffer_steps_per_env)
    mini_batch_size = batch_size // num_agent_training_mini_batches

    # Recording start time
    start_datetime = datetime.now()
    start_datetime_timestamp_str = start_datetime.strftime("%Y_%m_%d_%H_%M_%S")

    # Creating replay buffer for storing transition for training
    replay_buffer = ReplayBuffer(num_replay_buffer_steps_per_env, 
                                 envs.num_envs, 
                                 envs.envs[0].raw_observation_space, 
                                 envs.single_observation_space, 
                                 envs.single_action_space)

    # Setting up Tensorboard for saving training stats if requested
    if save_training_stats:
        tensorboard_writer = SummaryWriter(f"logs/doom_basic_level/rlhf_training_{start_datetime_timestamp_str}")

    # Initializing variables for tracking the training process 
    global_step = 0
    agent_training_iteration = 0 
    observations, infos = envs.reset()
    terminations = [0 for _ in range(envs.num_envs)]
    best_average_return = float("-inf")
    reward_sums = np.zeros(envs.num_envs, dtype=np.float32)
    returns = []

    while True:
        show_loading_screen(f"Training PPO Agent...")
        
        for step in range(0, num_replay_buffer_steps_per_env):
            global_step += envs.num_envs

            # Getting next action and it's value
            actions, log_probs, probs, values = agent.forward(observations)
            values = values.flatten()
            
            # Performing actions in the environments
            observations_, _, terminations_, _, infos = envs.step(actions)

            # Predicting reward for the observations and the corresponding actions
            rewards = reward_predictor.forward(observations)
            reward_sums = reward_sums + rewards

            # Saving transitions in replay buffer
            replay_buffer[step] = (
                np.stack(infos["raw_observations"]),
                observations,
                actions,
                log_probs,
                rewards,
                values,
                terminations
            )

            # Saving new observation and termination status for next step
            observations = observations_
            terminations = terminations_

            # Record episodic returns
            for index, done in enumerate(terminations):
                if done == 1:
                    reward_sum = reward_sums[index]
                    returns.append(reward_sums[index])
                    log_info(f"global_step={global_step}, episodic_return={reward_sums[index]}")

                    # Resetting rewards sum
                    reward_sums[index] = 0

        # Checking if the current mean is higher than previous highest mean and saving the model
        current_mean_episodic_return = np.mean(returns)
        log_info(f"Current Mean Episodic Return = {current_mean_episodic_return}")
        if current_mean_episodic_return > best_average_return:
            # Saving the model
            model_save_path = f"./models/rlhf_pipeline/training_run_{start_datetime_timestamp_str}/doom_ppo_agent/checkpoint_step_{global_step}"
            log_info(f"Saving models to `{model_save_path}`...")
            agent.save_models(model_save_path)
            log_info(f"Successfully saved models to `{model_save_path}`!")

            # Saving new best average return and clearing returns arrays
            best_average_return = current_mean_episodic_return
            returns.clear()

        # Training the agent
        agent_training_stats = agent.train(
            replay_buffer=replay_buffer,
            mini_batch_size=mini_batch_size,
            num_training_epochs=num_agent_training_epochs,
        )
        agent_training_iteration += 1
        log_info(f"SPS: {int(global_step / (time.time() - start_time))}")

        if save_training_stats:
            # tensorboard_writer.add_scalar("charts/learning_rate", training_stats.learning_rate, global_step)
            tensorboard_writer.add_scalar("ppo_agent/losses/value_loss", agent_training_stats.value_loss, global_step)
            tensorboard_writer.add_scalar("ppo_agent/losses/policy_loss", agent_training_stats.policy_loss, global_step)
            tensorboard_writer.add_scalar("ppo_agent/losses/entropy_loss", agent_training_stats.entropy_loss, global_step)
            tensorboard_writer.add_scalar("ppo_agent/charts/old_approx_kl", agent_training_stats.old_approx_kl, global_step)
            tensorboard_writer.add_scalar("ppo_agent/charts/approx_kl", agent_training_stats.approx_kl, global_step)
            tensorboard_writer.add_scalar("ppo_agent/charts/clip_fraction", agent_training_stats.clip_fraction, global_step)
            tensorboard_writer.add_scalar("ppo_agent/charts/explained_variance", agent_training_stats.explained_variance, global_step)
            tensorboard_writer.add_scalar("ppo_agent/charts/SPS", int(global_step / (time.time() - start_time)), global_step)

        if agent_training_iteration % human_feedback_interval == 0:
            # Preparing trajectories and videos for asking user for preference
            show_loading_screen("Preparing trajectories for asking user for preference...")
            segments = replay_buffer.get_segments(segment_length=num_steps_in_trajectory_video)
            trajectory_1, trajectory_2 = random.sample(segments, k=2)
            generate_video_from_doom_play_segments(trajectory_1, './temp/trajectory_1.mp4')
            generate_video_from_doom_play_segments(trajectory_2, './temp/trajectory_2.mp4')
            
            # Asking user for their preference
            user_preference = await ask_user_for_preference('./temp/trajectory_1.mp4', './temp/trajectory_2.mp4')
            log_info(f"user_preference = {user_preference}")

            # Training reward predictor based on the user preference
            show_loading_screen("Training reward predictor...")
            try:
                reward_predictor_training_stats = reward_predictor.train(trajectory_1, trajectory_2, user_preference)
                log_info(f"reward_predictor_training_loss={reward_predictor_training_stats['loss']}")

                if save_training_stats:
                    # Writing loss value to tensorboard
                    tensorboard_writer.add_scalar("ppo_agent/losses/loss", reward_predictor_training_stats["loss"], global_step)
            except Exception as error:
                # Catch the error and print the entire stack trace
                traceback.print_exc()

async def run_training_pipeline(envs: gym.vector.SyncVectorEnv, 
                agent: DoomPpoAgent, 
                reward_predictor: DoomHumanPreferenceRewardPredictor,
                num_replay_buffer_steps_per_env: int,
                num_pre_training_preference_requests: int,
                num_agent_training_mini_batches: int,
                num_agent_training_epochs: int,
                num_steps_in_trajectory_video: int,
                save_training_stats: bool = False):
    await pre_train(envs, 
                    agent, 
                    reward_predictor, 
                    num_replay_buffer_steps_per_env=num_replay_buffer_steps_per_env,
                    num_preference_requests=num_pre_training_preference_requests,
                    num_steps_in_trajectory_video=num_steps_in_trajectory_video)

    # await train(envs, 
    #             agent, 
    #             reward_predictor, 
    #             num_replay_buffer_steps_per_env=num_replay_buffer_steps_per_env, 
    #             num_agent_training_mini_batches=num_agent_training_mini_batches,
    #             num_agent_training_epochs=num_agent_training_epochs,
    #             save_training_stats=save_training_stats)

In [4]:
%gui asyncio

from utils.env import make_vizdoom_env

# Displaying UI and showing only the loading screen
clear_output()
display(layout)
hide_all_screens()

# Initializing environments with only the first enviroment rendering
num_envs = 8
envs = gym.vector.SyncVectorEnv([ make_vizdoom_env('envs/vizdoom/scenarios/basic.cfg', render_mode="human" if i == 0 else None) for i in range(num_envs)])

# Setting up agent and reward predictor
agent = DoomPpoAgent(envs.single_observation_space,
                     envs.single_action_space,
                     learning_rate=0.0001,
                     use_gpu=True)
reward_predictor = DoomHumanPreferenceRewardPredictor(envs.single_observation_space, 
                                                      envs.single_action_space,
                                                      learning_rate=1e-4,
                                                      use_gpu=True)

# Starting training process
event_loop = asyncio.get_event_loop()
training_task = event_loop.create_task(run_training_pipeline(envs, 
                                                             agent, 
                                                             reward_predictor, 
                                                             num_replay_buffer_steps_per_env=256, 
                                                             num_pre_training_preference_requests=500,
                                                             num_agent_training_mini_batches=32,
                                                             num_agent_training_epochs=10,
                                                             num_steps_in_trajectory_video=64,
                                                             save_training_stats=False))

HBox()

/Users/harish/opt/anaconda3/envs/doom-rlhf/lib/python3.9/site-packages/gymnasium/core.py:297: UserWarning: WARN: env.num_envs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.num_envs` for environment variables or `env.get_attr('num_envs')` that will search the reminding wrappers.
  logger.warn(


TypeError: empty() received an invalid combination of arguments - got (tuple, dtype=NoneType, device=NoneType), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.memory_format memory_format, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.memory_format memory_format, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
